# Boilerplate

There's probably a better way than what follows

In [ ]:
import sys
sys.path.insert(0, 'venv/lib/python3.10/site-packages')
import asyncio

from nwv2_python_wrapper import *
import nwv2_python_wrapper

# Run

What are all the Python wrappers available for us to import?

In [ ]:
[f for f in dir(nwv2_python_wrapper) if not f.startswith('__') and f != 'nwv2_python_wrapper']

## Transport Setup

Create a new transport interface, asynchronously. This uses the Python `asyncio` event loop, which a Jupyter Notebook already has running.

In [ ]:
tiface_fut = new_transport_interface("0.0.0.0", 0, TransportModeW("client"))
tiface_fut

Await the future to get the `TransportInterface`. Note: unlike in Rust, `await` isn't necessary to drive Future completion.

In [ ]:
tiface = await tiface_fut
tiface

### Run the Transport in background
Here is the fun part :)

In [ ]:
run_fut = tiface.run()
run_fut

If the Future completed right away, it means something's fishy.

In [ ]:
assert not run_fut.done()

Note the `transport: None` at the beginning of the following `repr` output. It is `None` because `run()` takes the `Transport`.

In [ ]:
tiface

## Transport Operation

### New Remote Endpoint

Make a transport command

In [ ]:
chococat_endpoint = EndpointW("157.230.134.224:2016")
chococat_endpoint

The endpoint will timeout after 30 seconds of inactivity (30000ms).

In [ ]:
new_endpoint_transport_cmd = TransportCmdW("newendpoint", endpoint=chococat_endpoint, timeout=30000)
new_endpoint_transport_cmd

Send the command to the Transport layer and get the response from it.

In [ ]:
await tiface.command_response(new_endpoint_transport_cmd)

### Send Packet to Remote

Create a `PacketSettings` wrapper. Note that the unique ID `tid` is optional, and duration is in milliseconds.

In [ ]:
packet_settings = PacketSettingsW(5000) # retry after 5000ms
packet_settings

Make another transport command

Make a packet

In [ ]:
getstatus_packet = PacketW(variant="getstatus", ping_nonce=98765432)
getstatus_packet

In [ ]:


send_packets_cmd = TransportCmdW("sendpackets",
                                 endpoint=chococat_endpoint,
                                 packet_infos=[packet_settings],
                                 packets=[getstatus_packet])
send_packets_cmd

SEND IT :)

In [ ]:
await tiface.command_response(send_packets_cmd)

Get all available notifications from the Transport layer.

In [ ]:
notifs = tiface.get_notifications()
notifs

Out of the above, list only packets delivered from the server.

In [ ]:
list(filter(lambda notif: notif.variant().lower() == 'packetdelivery', notifs))

Drop the packet so it doesn't keep getting resent.

In [ ]:
drop_packet_cmd = TransportCmdW("droppacket", endpoint=chococat_endpoint, tid=packet_settings.tid)
await tiface.command_response(drop_packet_cmd)

An alternative to the above is to drop all the packets with `TransportCmd::CancelTransmitQueue`.

In [ ]:
await tiface.command_response(TransportCmdW("canceltransmitqueue", endpoint=chococat_endpoint))

Drain the notifications queue.

In [ ]:
notifs = tiface.get_notifications()
notifs

In [ ]:
#notifs = tiface.get_notifications()
#notifs

In [ ]:
#notifs = tiface.get_notifications()
#notifs

Get the local address. This is useful for testing client and server interactions and the server port is 0, meaning the OS picks the port. By using `local_addr`, you can find out what the client should connect to.

In [ ]:
local_addr = tiface.local_addr
local_addr

# Shutdown

In [ ]:
# Commented so that you can run your own commands for the Transport layer after Restart and Run All
#await tiface.command_response(TransportCmdW("shutdown"))